### Setup

In [1]:
import pandas as pd
import requests

# Utils
import json
from pandas import json_normalize
import enum

# Options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

headers = {'Content-Type': 'application/json'}

In [2]:
#import os
#from google.cloud import secretmanager
#secrets = secretmanager.SecretManagerServiceClient()

#resource_name = f"{os.environ['KNADA_TEAM_SECRET']}/versions/latest"
#secret = secrets.access_secret_version(name=resource_name)
#data = secret.payload.data.decode('UTF-8')

### Config

In [3]:
class TeamType(enum.Enum):
    PRODUCT = "Produktteam"
    IT = "IT-team"
    ADMINISTRATION = "Forvaltningsteam"
    PROJECT = "Prosjektteam"
    OTHER = "Annet"
    UNKNOWN = "Ukjent"

In [4]:
class AreaType(enum.Enum):
    IT = "IT-område"
    PRODUCT_AREA = "Produktområde"
    PROJECT = "Prosjekt"
    OTHER = "Annet"

### Get Data

In [5]:
df_productarea = pd.DataFrame()
url = 'https://teamkatalog-api.nais.adeo.no/productarea'
r = requests.get(url, headers=headers)
content = r.json()['content']
normalized = json_normalize(content)
df_productarea = df_productarea.append(normalized, sort=True)
df_productarea['member-list'] = df_productarea['members'].apply(lambda x: json_normalize(x).to_records())

/tmp/ipykernel_242/1016234608.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_productarea = df_productarea.append(normalized, sort=True)


In [6]:
df_productarea.head(1)

,areaType,changeStamp.lastModifiedBy,changeStamp.lastModifiedDate,defaultArea,description,id,links.slackChannels,links.ui,members,name,paOwnerGroup,paOwnerGroup.ownerGroupMemberResourceList,paOwnerGroup.ownerResource,paOwnerGroup.ownerResource.email,paOwnerGroup.ownerResource.endDate,paOwnerGroup.ownerResource.familyName,paOwnerGroup.ownerResource.fullName,paOwnerGroup.ownerResource.givenName,paOwnerGroup.ownerResource.links.ui,paOwnerGroup.ownerResource.navIdent,paOwnerGroup.ownerResource.onLeave,paOwnerGroup.ownerResource.resourceType,paOwnerGroup.ownerResource.stale,paOwnerGroup.ownerResource.startDate,slackChannel,status,tags,member-list
0,PRODUCT_AREA,"L105681 - Lied, Lene",2022-05-07T11:48:31.0226,False,I produktområde helse utvikler vi digitale tje...,6b149078-927b-4570-a1ce-97bbb9499fb6,"[{'name': '#område-helse', 'url': 'https://sla...",https://teamkatalog.nav.no/area/6b149078-927b-...,"[{'navIdent': 'M164897', 'description': None, ...",Produktområde helse,NaN,"[{'navIdent': 'W138575', 'givenName': 'Kristin...",NaN,Kristian.Munthe@nav.no,NaN,Munthe,Kristian Munthe,Kristian,https://teamkatalog.nav.no/resource/M106440,M106440,False,INTERNAL,False,2000-12-28,#område-helse,ACTIVE,[],"[[0, M164897, None, [AREA_LEAD], 0, None, None..."


In [7]:
df_productarea = df_productarea.drop(columns=['changeStamp.lastModifiedBy'])

In [8]:
df_productarea.dtypes

areaType                                      object
changeStamp.lastModifiedDate                  object
defaultArea                                     bool
description                                   object
id                                            object
links.slackChannels                           object
links.ui                                      object
members                                       object
name                                          object
paOwnerGroup                                 float64
paOwnerGroup.ownerGroupMemberResourceList     object
paOwnerGroup.ownerResource                   float64
paOwnerGroup.ownerResource.email              object
paOwnerGroup.ownerResource.endDate           float64
paOwnerGroup.ownerResource.familyName         object
paOwnerGroup.ownerResource.fullName           object
paOwnerGroup.ownerResource.givenName          object
paOwnerGroup.ownerResource.links.ui           object
paOwnerGroup.ownerResource.navIdent           

In [9]:
def convert_area_type(area_type):
    if area_type:
        return AreaType[area_type].value
    else:
        return AreaType.OTHER.value

In [10]:
df_productarea["areaType"] = df_productarea["areaType"].apply(convert_area_type)

In [11]:
df_productarea.columns = df_productarea.columns.str.replace('.', '_')

/tmp/ipykernel_242/3703384248.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_productarea.columns = df_productarea.columns.str.replace('.', '_')


In [12]:
df_teams = pd.DataFrame()
url = 'https://teamkatalog-api.nais.adeo.no/team'
r = requests.get(url, headers=headers)
content = r.json()['content']
normalized = json_normalize(content)
df_teams = df_teams.append(normalized, sort=True)
df_teams['member-list'] = df_teams['members'].apply(lambda x: json_normalize(x).to_records())

/tmp/ipykernel_242/1674685619.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_teams = df_teams.append(normalized, sort=True)


In [13]:
df_teams.head(1)

,changeStamp.lastModifiedBy,changeStamp.lastModifiedDate,clusterIds,contactAddresses,contactPersonIdent,description,id,links.slackChannels,links.ui,members,naisTeams,name,officeHours,officeHours.days,officeHours.information,officeHours.location.code,officeHours.location.description,officeHours.location.displayName,officeHours.location.parent.code,officeHours.location.parent.description,officeHours.location.parent.displayName,officeHours.location.parent.parent.code,officeHours.location.parent.parent.description,officeHours.location.parent.parent.displayName,officeHours.location.parent.parent.parent,officeHours.location.parent.parent.type,officeHours.location.parent.type,officeHours.location.type,productAreaId,qaTime,slackChannel,status,tags,teamOwnerIdent,teamOwnershipType,teamType,member-list
0,"T160428 - Thrap-Meyer, Julian Ravn",2023-02-07T13:19:34.049889,[7916e6cc-e7ab-41d2-bdaf-33488766d702],[],J157811,Ansvar for nais deploy og de fleste av feature...,fc44e424-f5ae-4d82-bfd4-bb2f49cfb2ce,"[{'name': '#nais', 'url': 'https://slack.com/a...",https://teamkatalog.nav.no/team/fc44e424-f5ae-...,"[{'navIdent': 'E154363', 'description': None, ...",[],nais-features,NaN,[],None,FA1-BA-E8,Etasje 8,"Fyrstikkalléen 1, Bygg A, Etasje 8",FA1-BA,Bygg A,"Fyrstikkalléen 1, Bygg A",FA1,Fyrstikkalléen 1,Fyrstikkalléen 1,NaN,BUILDING,SECTION,FLOOR,6dbeedea-b23e-4bf7-a1cb-21d02d15e452,2022-11-21T13:49:00,#nais,ACTIVE,[],None,IT,PLATFORM,"[[0, E154363, None, [DEVELOPER], 0, None, None..."


In [14]:
df_teams.dtypes

changeStamp.lastModifiedBy                         object
changeStamp.lastModifiedDate                       object
clusterIds                                         object
contactAddresses                                   object
contactPersonIdent                                 object
description                                        object
id                                                 object
links.slackChannels                                object
links.ui                                           object
members                                            object
naisTeams                                          object
name                                               object
officeHours                                       float64
officeHours.days                                   object
officeHours.information                            object
officeHours.location.code                          object
officeHours.location.description                   object
officeHours.lo

In [15]:
df_teams = df_teams.drop(columns=['changeStamp.lastModifiedBy'])

In [16]:
def convert_team_type(team_type):
    return TeamType[team_type].value

In [17]:
df_teams.columns = df_teams.columns.str.replace('.', '_')

/tmp/ipykernel_242/2535767319.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_teams.columns = df_teams.columns.str.replace('.', '_')


### Publish data to BigQuery

In [18]:
#Navn på prosjekt og dataset som definert i BigQuery
project_id = 'org-prod-1016'
dataset = 'teamkatalogen'

from google.cloud import bigquery
client = bigquery.Client(project=project_id)

In [19]:
df_teams_pub= df_teams.copy()

In [20]:
df_teams_pub = df_teams_pub.astype({ 'clusterIds': 'string','links_slackChannels': 'string', 'members': 'string', 'member-list': 'string'})

In [21]:
#Istedenfor å spesifisere typen på variablene så plukker vi ut én kolonne til å teste med.
#df_teams_pub = df_teams_pub[['name']]

In [22]:
#df_teams_pub[['name']]

In [23]:
#df_teams_pub = df_teams_pub.head(1)

In [24]:
table = 'teams'
destination_table = f"{project_id}.{dataset}.{table}"

job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE",)
job = client.load_table_from_dataframe(df_teams_pub, destination_table, job_config=job_config)
job.result()

/home/jovyan/.local/lib/python3.9/site-packages/google/cloud/bigquery/_pandas_helpers.py:571: UserWarning: Pyarrow could not determine the type of columns: contactAddresses.
  warnings.warn(


LoadJob<project=org-prod-1016, location=europe-north1, id=0cd13a12-0a90-4ae7-bb23-3be06a102c23>